# parametre du pilote et librairie du container docker

fonctionne mais il faut le parametrer dans le container docker
en ajoutant dans le docker file les parametres suivant :

In [ ]:
# Installation des dépendances pour Chrome
RUN apt-get update && apt-get install -y \
    wget \
    curl \
    unzip \
    gnupg2 \
    xvfb \
    libxi6 \
    libgconf-2-4 \
    libnss3 \
    libgbm1 \
    libasound2 \
    libxss1 \
    fonts-liberation \
    libappindicator3-1 \
    libatk-bridge2.0-0 \
    libatspi2.0-0 \
    libgtk-3-0 \
    libx11-xcb1 \
    xdg-utils \
    && rm -rf /var/lib/apt/lists/*

# Installation de Chrome
RUN wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add - \
    && echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list \
    && apt-get update \
    && apt-get install -y google-chrome-stable \
    && rm -rf /var/lib/apt/lists/*

# Installation de ChromeDriver correspondant à Chrome 130
RUN wget -q "https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/130.0.6723.69/linux64/chromedriver-linux64.zip" \
    && unzip chromedriver-linux64.zip \
    && mv chromedriver-linux64/chromedriver /usr/local/bin/ \
    && rm -rf chromedriver-linux64.zip chromedriver-linux64 \
    && chmod +x /usr/local/bin/chromedriver

# Configuration des permissions
RUN chmod -R 777 /root/.cache

# cadremploi (En chantier)

a retravailler avec selenium en utilisant la meme logique que pour welcome to the jungle

In [ ]:
//*[@id="liste-postuler"]/div[1]/li/a[1]
//*[@id="liste-postuler"]/div[1]/li/a[1]
//*[@id="liste-postuler"]/div[3]/li/a[1]
//*[@id="liste-postuler"]/div[6]/li/a[1]
//*[@id="liste-postuler"]/div[8]/li/a[1]

//*[@id="liste-postuler"]/b/div[22]/li/a[1]
//*[@id="liste-postuler"]/b/div[26]/li/a[1]

# pour recuper l'url d'une offre specifique :
<a class="row js-lien-detail-offre offre" href="/emploi/detail_offre?offreId=156453275908950362"

# pour la date de publication :
<time class="date-publication"

SyntaxError: invalid syntax (<ipython-input-2-2e5d9cadf535>, line 1)

In [ ]:
if 'data_loader':
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service
    import pandas as pd
    import time
    import re

    @data_loader
    def load_data(*args, **kwargs):
        """
        Fonction de chargement des données pour Mage
        """
        def parse_time_ago(time_text):
            if 'heures' in time_text or 'heure' in time_text:
                hours = int(re.search(r'\d+', time_text).group())
                return hours
            elif 'minutes' in time_text:
                minutes = int(re.search(r'\d+', time_text).group())
                return minutes / 60
            else:
                return 24

        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.binary_location = '/usr/bin/google-chrome'

        try:
            service = Service(executable_path='/usr/local/bin/chromedriver')
            driver = webdriver.Chrome(service=service, options=options)

            url = "https://www.cadremploi.fr/emploi/liste_offres?locLibre=ile%20de%20france&motscles=data"

            driver.get(url)

            wait = WebDriverWait(driver, 10)
            base_xpath = '//*[@id="app"]/div/div/div/div[2]/div/ul/li'
            wait.until(EC.presence_of_element_located((By.XPATH, base_xpath)))

            jobs_data = []
            should_continue_scrolling = True

            while should_continue_scrolling:
                visible_elements = []
                i = 1
                while True:
                    try:
                        element_xpath = f'//*[@id="app"]/div/div/div/div[2]/div/ul/li[{i}]'
                        element = driver.find_element(By.XPATH, element_xpath)
                        visible_elements.append(element)
                        i += 1
                    except:
                        break

                for element in visible_elements:
                    try:
                        url_element = element.find_element(By.XPATH, './/div/div/div/div[2]/a')
                        url = url_element.get_attribute('href')

                        time_element = element.find_element(By.XPATH, './/time/span')
                        time_ago = time_element.text

                        hours_ago = parse_time_ago(time_ago)

                        if hours_ago < 24:
                            jobs_data.append({
                                'url': url,
                                'published': time_ago,
                                'hours_ago': hours_ago
                            })
                        else:
                            should_continue_scrolling = False
                            break

                    except Exception as e:
                        continue

                if should_continue_scrolling:
                    last_height = driver.execute_script("return document.body.scrollHeight")
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)
                    new_height = driver.execute_script("return document.body.scrollHeight")

                    if new_height == last_height:
                        break

            df = pd.DataFrame(jobs_data)
            return df

        except Exception as e:
            import traceback
            print(traceback.format_exc())
            return pd.DataFrame()

        finally:
            if 'driver' in locals():
                driver.quit()

# WTTJ



*   recuperation des URL d'offre de la premiére page :



In [ ]:
if 'data_loader':
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service
    import pandas as pd
    import time
    import re

    @data_loader
    def load_data(*args, **kwargs):
        """
        Fonction de chargement des données pour Mage
        """
        def parse_time_ago(time_text):
            if 'heures' in time_text or 'heure' in time_text:
                hours = int(re.search(r'\d+', time_text).group())
                return hours
            elif 'minutes' in time_text:
                minutes = int(re.search(r'\d+', time_text).group())
                return minutes / 60
            else:
                return 24

        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.binary_location = '/usr/bin/google-chrome'

        try:
            service = Service(executable_path='/usr/local/bin/chromedriver')
            driver = webdriver.Chrome(service=service, options=options)

            url = "https://www.welcometothejungle.com/fr/jobs?query=data&refinementList%5Bcontract_type%5D%5B%5D=internship&refinementList%5Bcontract_type%5D%5B%5D=temporary&refinementList%5Bcontract_type%5D%5B%5D=apprenticeship&refinementList%5Boffices.state%5D%5B%5D=%C3%8Ele-de-France&refinementList%5Boffices.country_code%5D%5B%5D=FR&page=1&aroundQuery=%C3%8Ele-de-France%2C%20France&sortBy=mostRelevant"

            driver.get(url)

            wait = WebDriverWait(driver, 10)
            base_xpath = '//*[@id="app"]/div/div/div/div[2]/div/ul/li'
            wait.until(EC.presence_of_element_located((By.XPATH, base_xpath)))

            jobs_data = []
            should_continue_scrolling = True

            while should_continue_scrolling:
                visible_elements = []
                i = 1
                while True:
                    try:
                        element_xpath = f'//*[@id="app"]/div/div/div/div[2]/div/ul/li[{i}]'
                        element = driver.find_element(By.XPATH, element_xpath)
                        visible_elements.append(element)
                        i += 1
                    except:
                        break

                for element in visible_elements:
                    try:
                        url_element = element.find_element(By.XPATH, './/div/div/div/div[2]/a')
                        url = url_element.get_attribute('href')

                        time_element = element.find_element(By.XPATH, './/time/span')
                        time_ago = time_element.text

                        hours_ago = parse_time_ago(time_ago)

                        if hours_ago < 24:
                            jobs_data.append({
                                'url': url,
                                'published': time_ago,
                                'hours_ago': hours_ago
                            })
                        else:
                            should_continue_scrolling = False
                            break

                    except Exception as e:
                        continue

                if should_continue_scrolling:
                    last_height = driver.execute_script("return document.body.scrollHeight")
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)
                    new_height = driver.execute_script("return document.body.scrollHeight")

                    if new_height == last_height:
                        break

            df = pd.DataFrame(jobs_data)
            return df

        except Exception as e:
            import traceback
            print(traceback.format_exc())
            return pd.DataFrame()

        finally:
            if 'driver' in locals():
                driver.quit()



*   transformer pour recuperer les elements que je veut sur chaque url



In [ ]:
if 'transformer':
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options
    from webdriver_manager.chrome import ChromeDriverManager
    import pandas as pd
    import time
    from datetime import datetime
    import pytz

    @transformer
    def transform_df(df, *args, **kwargs):
        """
        Extrait les informations détaillées des offres d'emploi
        """
        if df is None or df.empty:
            print("Aucune URL à traiter")
            return pd.DataFrame()

        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')

        paris_tz = pytz.timezone('Europe/Paris')
        current_date = datetime.now(paris_tz).strftime("%d-%m-%Y")

        data = {
            'entreprise': [],
            'publication': [],
            'poste': [],
            'contrat': [],
            'profil': [],
            'description': [],
            'ville': [],
            'lien': [],
            'source': []
        }

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        wait = WebDriverWait(driver, 10)

        try:
            total_urls = len(df)

            for index, row in df.iterrows():
                url = row['url']

                try:
                    driver.get(url)
                    time.sleep(2)

                    def safe_extract(xpath, attribute=None):
                        try:
                            element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
                            return element.get_attribute(attribute) if attribute else element.text
                        except Exception:
                            return "Non disponible"

                    data['entreprise'].append(safe_extract('//*[@id="app"]/div/div/div/div/div[3]/section/div[1]/div[1]/a/div/span'))
                    data['publication'].append(current_date)
                    data['poste'].append(safe_extract('//*[@id="app"]/div/div/div/div/div[3]/section/div[1]/h2'))
                    data['contrat'].append(safe_extract('//*[@id="app"]/div/div/div/div/div[3]/section/div[1]/div[2]/div/div/div[1]'))
                    data['profil'].append(safe_extract('//*[@id="the-position-section"]/div/div[2]/div[2]/div/div[1]'))
                    data['description'].append(safe_extract('//*[@id="the-position-section"]/div/div[2]/div[1]/div/div[1]'))
                    data['ville'].append(safe_extract('//*[@id="app"]/div/div/div/div/div[3]/section/div[1]/div[2]/div/div/div[2]/span/span'))
                    data['lien'].append(url)
                    data['source'].append("Welcome to the Jungle")

                except Exception as e:
                    continue

                time.sleep(1)

            extracted_df = pd.DataFrame(data)
            return extracted_df

        except Exception as e:
            return pd.DataFrame()

        finally:
            driver.quit()

In [ ]:
import pandas as pd
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@transformer
def clean_dataframe(df, *args, **kwargs):
    """
    Nettoie le DataFrame
    """
    df_clean = df.copy()
    df_clean['publication'] = pd.to_datetime(df_clean['publication'], dayfirst=True, errors='coerce')
    df_clean['publication'] = df_clean['publication'].dt.strftime('%d-%m-%Y')
    df_clean = df_clean.drop_duplicates()
    return df_clean


@test
def test_output(output, *args) -> None:
    """
    Tests de validation
    """
    assert output is not None, 'La sortie est non définie'
    assert len(output) > 0, 'Le DataFrame est vide'
    date_format = output['publication'].str.match(r'\d{2}-\d{2}-\d{4}')
    assert date_format.all(), 'Format de date incorrect'
    assert len(output) == len(output.drop_duplicates()), 'Il reste des doublons dans le DataFrame'



*   data exporter pour pouvoir stocker temporairement les données sur mon bucket minio  
  
le script va charger le fichier csv deja existant depuis le bucket, verifier les doublons avec les nouvelles données, puis concat les nouvelles offres dans le csv, pour le stocker

In [ ]:
import pandas as pd
from io import BytesIO
from minio import Minio
import os
from datetime import datetime
import pytz

@data_exporter
def export_to_minio(df, *args, **kwargs):
    """
    Exporte le DataFrame des offres vers MinIO en écrasant l'ancien fichier
    """
    client = Minio(
        "minio:9000",
        access_key=os.environ.get('MINIO_ACCESS_KEY'),
        secret_key=os.environ.get('MINIO_SECRET_KEY'),
        secure=False
    )

    bucket_name = "offresdata"
    file_name = "welcometothejungle.csv"

    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)
    df = df.drop_duplicates(subset=['lien'], keep='last')
    csv_bytes = df.to_csv(index=False).encode('utf-8')
    client.put_object(
        bucket_name,
        file_name,
        BytesIO(csv_bytes),
        length=len(csv_bytes),
        content_type='application/csv'
    )

    return f"✅ {len(df)} offres exportées"

@test
def test_output(output, *args) -> None:
    assert output is not None and isinstance(output, str), 'La sortie doit être une chaîne non vide'

# hellowork

va simplement recuperer les données de l'api hellowork  
en divisant le loader en trois :

*   CDD

*   Stage
*   Alternance




In [ ]:
# CDD

import io
import pandas as pd
import requests
import re
from tqdm import tqdm
if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

def api_helloworld_cdd(periode="h", local="ile de france", keyword="data", contrat="CDD"):
    results = []

    params = {
        "k": keyword,
        "k_autocomplete": "",
        "l": local,
        "l_autocomplete": "",
        "sort": "relevance",
        "c": contrat,
        "cod": "all",
        "ray": 20,
        "d": periode
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        "Referer": "www.hellowork.com"
    }

    response = requests.get("https://www.hellowork.com/searchoffers/getsearchfacets", params=params, headers=headers)
    response_json = response.json()

    if "Results" in response_json and response_json["Results"]:
        for result in tqdm(response_json["Results"]):
            publication_date = result["PublishDate"]
            match = re.search(r'(\d{4}-\d{2}-\d{2})', publication_date)
            formatted_date = match.group(0) if match else publication_date
            lien = result["UrlOffre"]
            lien_origine = f'https://www.hellowork.com{lien}'
            localisations = result.get("Localisations", [])
            ville = localisations[3]["Label"] if len(localisations) > 3 else "N/A"
            result_dict = {
                "entreprise": result["CompanyName"],
                "publication": formatted_date,
                "poste": result["OfferTitle"],
                "contrat": result["ContractType"],
                "profil": result["Profile"],
                "description": result["Description"],
                "ville": ville,
                "lien": lien_origine,
                "source": result["ResponseUrl"]
            }
            results.append(result_dict)

    df = pd.DataFrame(results)
    df.reset_index(drop=True, inplace=True)
    return df


@data_loader
def load_data_from_api(*args, **kwargs):
    """
    Charge les données depuis l'API HelloWork
    """
    df = api_helloworld_cdd(
        periode="h",  # h pour aujourd'hui
        local="ile de france",
        keyword="data",
        contrat="CDD"
    )
    return df


@test
def test_output(output, *args) -> None:
    """
    Test de la sortie du bloc
    """
    assert output is not None, 'La sortie est non définie'
    assert isinstance(output, pd.DataFrame), 'La sortie doit être un DataFrame'
    assert len(output.columns) > 0, 'Le DataFrame ne doit pas être vide'

In [ ]:
# alternance

import io
import pandas as pd
import requests
import re
from tqdm import tqdm
if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

def api_helloworld_alternance(periode="h", local="ile de france", keyword="data", contrat="Alternance"):
    results = []

    params = {
        "k": keyword,
        "k_autocomplete": "",
        "l": local,
        "l_autocomplete": "",
        "sort": "relevance",
        "c": contrat,
        "cod": "all",
        "ray": 20,
        "d": periode
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        "Referer": "www.hellowork.com"
    }

    response = requests.get("https://www.hellowork.com/searchoffers/getsearchfacets", params=params, headers=headers)
    response_json = response.json()

    if "Results" in response_json and response_json["Results"]:
        for result in tqdm(response_json["Results"]):
            publication_date = result["PublishDate"]
            match = re.search(r'(\d{4}-\d{2}-\d{2})', publication_date)
            formatted_date = match.group(0) if match else publication_date
            lien = result["UrlOffre"]
            lien_origine = f'https://www.hellowork.com{lien}'
            localisations = result.get("Localisations", [])
            ville = localisations[3]["Label"] if len(localisations) > 3 else "N/A"
            result_dict = {
                "entreprise": result["CompanyName"],
                "publication": formatted_date,
                "poste": result["OfferTitle"],
                "contrat": result["ContractType"],
                "profil": result["Profile"],
                "description": result["Description"],
                "ville": ville,
                "lien": lien_origine,
                "source": result["ResponseUrl"]
            }
            results.append(result_dict)

    df = pd.DataFrame(results)
    df.reset_index(drop=True, inplace=True)
    return df

@data_loader
def load_data_from_api(*args, **kwargs):
    """
    Template for loading data from API
    """
    df = api_helloworld_alternance(
        periode="h",  # h pour aujourd'hui
        local="ile de france",
        keyword="data",
        contrat="Alternance"
    )
    return df


@test
def test_output(output, *args) -> None:
    """
    Test de la sortie du bloc
    """
    assert output is not None, 'La sortie est non définie'
    assert isinstance(output, pd.DataFrame), 'La sortie doit être un DataFrame'
    assert len(output.columns) > 0, 'Le DataFrame ne doit pas être vide'

In [ ]:
# stage

import io
import pandas as pd
import requests
import re
from tqdm import tqdm
if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

def api_helloworld_stage(periode="h", local="ile de france", keyword="data", contrat="Stage"):
    results = []

    params = {
        "k": keyword,
        "k_autocomplete": "",
        "l": local,
        "l_autocomplete": "",
        "sort": "relevance",
        "c": contrat,
        "cod": "all",
        "ray": 20,
        "d": periode
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        "Referer": "www.hellowork.com"
    }

    response = requests.get("https://www.hellowork.com/searchoffers/getsearchfacets", params=params, headers=headers)
    response_json = response.json()

    if "Results" in response_json and response_json["Results"]:
        for result in tqdm(response_json["Results"]):
            publication_date = result["PublishDate"]
            match = re.search(r'(\d{4}-\d{2}-\d{2})', publication_date)
            formatted_date = match.group(0) if match else publication_date
            lien = result["UrlOffre"]
            lien_origine = f'https://www.hellowork.com{lien}'
            localisations = result.get("Localisations", [])
            ville = localisations[3]["Label"] if len(localisations) > 3 else "N/A"
            result_dict = {
                "entreprise": result["CompanyName"],
                "publication": formatted_date,
                "poste": result["OfferTitle"],
                "contrat": result["ContractType"],
                "profil": result["Profile"],
                "description": result["Description"],
                "ville": ville,
                "lien": lien_origine,
                "source": result["ResponseUrl"]
            }
            results.append(result_dict)

    df = pd.DataFrame(results)
    df.reset_index(drop=True, inplace=True)
    return df

@data_loader
def load_data_from_api(*args, **kwargs):
    """
    Template for loading data from API
    """
    df = api_helloworld_stage(
        periode="h",  # h pour aujourd'hui
        local="ile de france",
        keyword="data",
        contrat="Stage"
    )
    return df


@test
def test_output(output, *args) -> None:
    """
    Test de la sortie du bloc
    """
    assert output is not None, 'La sortie est non définie'
    assert isinstance(output, pd.DataFrame), 'La sortie doit être un DataFrame'
    assert len(output.columns) > 0, 'Le DataFrame ne doit pas être vide'

*   transformer  
qui va fusionner les 3 dataframe en un seul



In [ ]:
import pandas as pd
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer

@transformer
def merge_dataframes(df_cdd, df_cdi, df_stage, *args, **kwargs):
    """
    Fusionne les trois DataFrames
    """
    df_combined = pd.concat([df_cdd, df_cdi, df_stage], ignore_index=True)

    return df_combined


* transformer 2  
pour le nettoyage et la normalisation des données

In [ ]:
import pandas as pd
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@transformer
def clean_dataframe(df, *args, **kwargs):
    """
    Nettoie le DataFrame
    """
    df_clean = df.copy()
    df_clean['publication'] = pd.to_datetime(df_clean['publication'], dayfirst=True, errors='coerce')
    df_clean['publication'] = df_clean['publication'].dt.strftime('%d-%m-%Y')
    df_clean = df_clean.drop_duplicates()
    return df_clean


@test
def test_output(output, *args) -> None:
    """
    Tests de validation
    """
    assert output is not None, 'La sortie est non définie'
    assert len(output) > 0, 'Le DataFrame est vide'
    date_format = output['publication'].str.match(r'\d{2}-\d{2}-\d{4}')
    assert date_format.all(), 'Format de date incorrect'
    assert len(output) == len(output.drop_duplicates()), 'Il reste des doublons dans le DataFrame'

* exporter  
qui va stocker les données sur le bucket minio en chargant le fichier deja existant si il y"en a un, verifier les doublons et concat le tout.

In [ ]:
import pandas as pd
from io import BytesIO
from minio import Minio
import os
from datetime import datetime
import pytz

@data_exporter
def export_to_minio(df, *args, **kwargs):
    """
    Exporte le DataFrame des offres vers MinIO en écrasant l'ancien fichier
    """
    client = Minio(
        "minio:9000",
        access_key=os.environ.get('MINIO_ACCESS_KEY'),
        secret_key=os.environ.get('MINIO_SECRET_KEY'),
        secure=False
    )

    bucket_name = "offresdata"
    file_name = "hellowork.csv"

    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)
    df = df.drop_duplicates(subset=['lien'], keep='last')
    csv_bytes = df.to_csv(index=False).encode('utf-8')
    client.put_object(
        bucket_name,
        file_name,
        BytesIO(csv_bytes),
        length=len(csv_bytes),
        content_type='application/csv'
    )

    return f"{len(df)} offres exportées"

@test
def test_output(output, *args) -> None:
    assert output is not None and isinstance(output, str), 'La sortie doit être une chaîne non vide'

# Concatener l'ensembles des CSV du bucket minio :

* charger le csv hellowork :

In [ ]:
if 'data_loader':
    import pandas as pd
    from minio import Minio
    import io
    import os

    @data_loader
    def load_hellwork_from_minio(*args, **kwargs):
        """
        Charge le fichier CSV Hellowork depuis Minio
        """
        try:
            client = Minio(
                "minio:9000",
                access_key=os.environ.get('MINIO_ACCESS_KEY'),
                secret_key=os.environ.get('MINIO_SECRET_KEY'),
                secure=False
            )
            data = client.get_object("offresdata", "hellowork.csv")
            df = pd.read_csv(io.BytesIO(data.read()))
            return df

        except Exception as e:
            raise e

* charger le csv WTTJ :

In [ ]:
if 'data_loader':
    import pandas as pd
    from minio import Minio
    import io
    import os

    @data_loader
    def load_wttj_from_minio(*args, **kwargs):
        """
        Charge le fichier CSV Welcome to the Jungle depuis Minio
        """
        try:
            client = Minio(
                "minio:9000",
                access_key=os.environ.get('MINIO_ACCESS_KEY'),
                secret_key=os.environ.get('MINIO_SECRET_KEY'),
                secure=False
            )

            data = client.get_object("offresdata", "welcometothejungle.csv")
            df = pd.read_csv(io.BytesIO(data.read()))
            return df

        except Exception as e:
            raise e

* concatenner les CSV :

In [ ]:
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@transformer
def transform(df_hellowork, df_wttj, *args, **kwargs):
    """
    Fusionne les DataFrames avec les données existantes et sauvegarde dans Minio
    """
    from minio import Minio
    import pandas as pd
    import io
    import os

    client = Minio(
        "minio:9000",
        access_key=os.environ.get('MINIO_ACCESS_KEY'),
        secret_key=os.environ.get('MINIO_SECRET_KEY'),
        secure=False
    )

    bucket_name = "offresdata"
    file_name = "rag_data.csv"

    df_hellowork = pd.DataFrame(df_hellowork) if isinstance(df_hellowork, list) else df_hellowork
    df_wttj = pd.DataFrame(df_wttj) if isinstance(df_wttj, list) else df_wttj
    df_new = pd.concat([df_hellowork, df_wttj], ignore_index=True)
    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)
    df_merged = df_new
    if client.bucket_exists(bucket_name):
        objects = client.list_objects(bucket_name, prefix=file_name)
        if any(True for _ in objects):
            data = client.get_object(bucket_name, file_name)
            df_existing = pd.read_csv(io.BytesIO(data.read()))
            df_merged = pd.concat([df_existing, df_new], ignore_index=True)
            df_merged = df_merged.drop_duplicates()
    csv_buffer = io.BytesIO()
    df_merged.to_csv(csv_buffer, index=False, encoding='utf-8')
    csv_buffer.seek(0)
    client.put_object(
        bucket_name,
        file_name,
        csv_buffer,
        length=csv_buffer.getbuffer().nbytes,
        content_type='text/csv'
    )
    print(df_merged.info())
    return df_merged


@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'

* exporter les données vers le bucket minio :  
les données sont sauvegardé au format parquet.

In [ ]:
import pandas as pd
from io import BytesIO
from minio import Minio
import os
from datetime import datetime
import pytz

@data_exporter
def export_to_minio(df, *args, **kwargs):
    """
    Exporte le DataFrame des offres vers MinIO en écrasant l'ancien fichier
    """
    client = Minio(
        "minio:9000",
        access_key=os.environ.get('MINIO_ACCESS_KEY'),
        secret_key=os.environ.get('MINIO_SECRET_KEY'),
        secure=False
    )

    bucket_name = "offresdata"
    file_name = "rag_data.csv"

    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)
    df = df.drop_duplicates(subset=['lien'], keep='last')
    csv_bytes = df.to_csv(index=False).encode('utf-8')
    client.put_object(
        bucket_name,
        file_name,
        BytesIO(csv_bytes),
        length=len(csv_bytes),
        content_type='application/csv'
    )

    return f"{len(df)} offres exportées"

@test
def test_output(output, *args) -> None:
    assert output is not None and isinstance(output, str), 'La sortie doit être une chaîne non vide'




# nettoyage

In [ ]:
if 'data_loader':
    import pandas as pd
    from minio import Minio
    import io
    import os

    @data_loader
    def load_hellwork_from_minio(*args, **kwargs):
        try:
            client = Minio(
                "minio:9000",
                access_key=os.environ.get('MINIO_ACCESS_KEY'),
                secret_key=os.environ.get('MINIO_SECRET_KEY'),
                secure=False
            )
            data = client.get_object("offresdata", "rag_data.csv")
            df = pd.read_csv(io.BytesIO(data.read()))
            return df

        except Exception as e:
            raise e

In [ ]:
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

import pandas as pd
import re

@transformer
def transform_clean_poste(data, *args, **kwargs):
    def clean_job_title(title):
        if not isinstance(title, str):
            return "Poste Non Spécifié"
        cleaned = title.lower()

        patterns = [
            r"\b(stage|alternance|cdd|cdi|alternant|stagiaire|intern|intérim)\b",
            r"\b(bac\s*\+\s*\d+|fin\s+d['\']études|étudiant|formation|école|master|licence|mastère)\b",
            r"\b(paris|grenoble|lyon|france|île-de-france|idf)\b",
            r"\b(h/f|f/h|h-f|f-h|m/f/d)\b",
            r"[-–—\s]*[(]?(h/f|f/h|h-f|f-h|m/f/d)[)]?",
            r"\b(temps\s+plein|temps\s+partiel|débutant|confirmé|junior|senior)\b",
            r"\b(\d+\s*(mois|ans?|années?|months?|years?))\b",
            r"\b(deepki|ref\s*:\s*[^\s]+)\b",
            r"\b(ref|reference)\s*\.?\s*:?\s*[a-z0-9]+\b",
            r"^(de|en|pour|le|la|les|un|une|des)\s+",
            r"\s+(de|en|pour|le|la|les|un|une|des)$"
        ]

        cleaned = re.sub(r'\([^)]*\)', '', cleaned)
        for pattern in patterns:
            cleaned = re.sub(pattern, ' ', cleaned, flags=re.IGNORECASE)
        cleaned = re.sub(r'[,;:]', ' ', cleaned)
        cleaned = re.sub(r'[-–—]{2,}', ' ', cleaned)
        cleaned = re.sub(r'^[-–—]+|[-–—]+$', '', cleaned)
        cleaned = re.sub(r'\s+[-–—]+\s+', ' ', cleaned)
        cleaned = re.sub(r'\s+[-–—]\s+', ' ', cleaned)
        cleaned = re.sub(r'\s*[-–—]\s*$', '', cleaned)
        cleaned = re.sub(r'^\s*[-–—]\s*', '', cleaned)
        cleaned = re.sub(r'\s+', ' ', cleaned)
        cleaned = cleaned.strip()
        cleaned = ' '.join(word.capitalize() for word in cleaned.split())
        cleaned = re.sub(r'\s*[-–—]+$', '', cleaned)
        cleaned = re.sub(r'\d+', '', cleaned)

        return cleaned

    data['poste'] = data['poste'].apply(clean_job_title)

    return data

@test
def test_output(output, *args) -> None:
    assert output is not None, 'Le output ne peut pas être None'
    assert 'poste' in output.columns, 'La colonne poste doit être présente'
    assert output['poste'].isnull().sum() == 0, 'Il ne doit pas y avoir de valeurs nulles'

In [ ]:
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

import pandas as pd

@transformer
def transform_clean_contrat(data, *args, **kwargs):
    data['contrat'] = data['contrat'].astype(str).str.replace(r'\s*\(.*\)', '', regex=True).str.strip()
    data['contrat'] = data['contrat'].replace('CDD / Temporaire', 'CDD')

    return data

@test
def test_output(output, *args) -> None:
    assert output is not None, 'Le output ne peut pas être None'
    assert 'contrat' in output.columns, 'La colonne contrat doit être présente'
    assert not output['contrat'].str.contains(r'\(.*\)').any(), 'Il reste des parenthèses dans les contrats'
    assert not (output['contrat'] == 'CDD / Temporaire').any(), 'Il reste des valeurs non remplacées'

In [ ]:
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

import pandas as pd

@transformer
def transform_merge_description(data, *args, **kwargs):
    """
    Fusionne les colonnes description et profil, supprime les colonnes non nécessaires,
    et génère des statistiques sur le dataset.

    Args:
        data: Le DataFrame avec les colonnes à fusionner

    Returns:
        DataFrame: Le DataFrame avec les colonnes fusionnées et nettoyé
    """
    # Fusion des colonnes
    data['description_poste'] = (data['description'].fillna('').astype(str) +
                                '\n\n' +
                                data['profil'].fillna('').astype(str))

    # Nettoyage des cas où les deux colonnes étaient vides
    data['description_poste'] = data['description_poste'].replace('^\n\n$', '', regex=True)

    # Suppression des colonnes originales et de la colonne source
    data = data.drop(['description', 'profil', 'source'], axis=1)

    # Génération et affichage des statistiques
    print("\n=== Statistiques du Dataset ===\n")

    # Nombre total de lignes et colonnes
    print(f"Dimensions du DataFrame : {data.shape[0]} lignes × {data.shape[1]} colonnes\n")

    # Statistiques pour chaque colonne
    for column in data.columns:
        print(f"\n=== Statistiques pour la colonne '{column}' ===")

        # Nombre de valeurs manquantes
        missing_count = data[column].isna().sum()
        missing_percentage = (missing_count / len(data)) * 100
        print(f"Valeurs manquantes : {missing_count} ({missing_percentage:.2f}%)")

        # Nombre de valeurs uniques
        unique_count = data[column].nunique()
        print(f"Valeurs uniques : {unique_count}")

        # Affichage des valeurs uniques pour les colonnes catégorielles
        if data[column].dtype == 'object' and unique_count < 20:  # Limite à 20 valeurs uniques
            print("Valeurs uniques et leurs fréquences :")
            value_counts = data[column].value_counts().head(10)  # Top 10 des valeurs les plus fréquentes
            for value, count in value_counts.items():
                percentage = (count / len(data)) * 100
                print(f"  - {value}: {count} occurrences ({percentage:.2f}%)")

        print("\n" + "="*50)

    return data

@test
def test_output(output, *args) -> None:
    """
    Test la validité du DataFrame de sortie.
    """
    assert output is not None, 'Le output ne peut pas être None'
    assert 'description_poste' in output.columns, 'La colonne description_poste doit être présente'
    assert 'description' not in output.columns, 'La colonne description doit être supprimée'
    assert 'profil' not in output.columns, 'La colonne profil doit être supprimée'
    assert 'source' not in output.columns, 'La colonne source doit être supprimée'
    # Vérifie qu'il n'y a pas que des \n\n dans la colonne description_poste
    assert not (output['description_poste'] == '\n\n').any(), 'Il reste des lignes vides non nettoyées'

In [ ]:
import pandas as pd
from io import BytesIO
from minio import Minio
import os
from datetime import datetime
import pytz

@data_exporter
def export_to_minio(df, *args, **kwargs):
    """
    Exporte le DataFrame des offres vers MinIO en écrasant l'ancien fichier
    """
    client = Minio(
        "minio:9000",
        access_key=os.environ.get('MINIO_ACCESS_KEY'),
        secret_key=os.environ.get('MINIO_SECRET_KEY'),
        secure=False
    )

    bucket_name = "offresdata"
    file_name = "rag_data_clean.csv"

    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)
    df = df.drop_duplicates(subset=['lien'], keep='last')
    csv_bytes = df.to_csv(index=False).encode('utf-8')
    client.put_object(
        bucket_name,
        file_name,
        BytesIO(csv_bytes),
        length=len(csv_bytes),
        content_type='application/csv'
    )

    return f"{len(df)} offres exportées"

@test
def test_output(output, *args) -> None:
    assert output is not None and isinstance(output, str), 'La sortie doit être une chaîne non vide'

# export vers postgresML

In [ ]:
if 'data_loader':
    import pandas as pd
    from minio import Minio
    import io
    import os

    @data_loader
    def load_hellwork_from_minio(*args, **kwargs):
        """
        Charge le fichier CSV Hellwork depuis Minio
        """
        try:
            client = Minio(
                "minio:9000",
                access_key=os.environ.get('MINIO_ACCESS_KEY'),
                secret_key=os.environ.get('MINIO_SECRET_KEY'),
                secure=False
            )
            data = client.get_object("offresdata", "rag_data_clean.csv")
            df = pd.read_csv(io.BytesIO(data.read()))
            print(df.dtypes)
            return df

        except Exception as e:
            raise e

In [ ]:
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
    from mage_ai.data_preparation.shared.secrets import get_secret_value
import pandas as pd
from datetime import date

@transformer
def transform_data(df, *args, **kwargs):
    """
    Prépare les données pour l'insertion dans une base SQL
    """
    df_clean = df.copy()
    df_clean['publication'] = pd.to_datetime(df_clean['publication'], format='%d-%m-%Y').dt.strftime('%d/%m/%Y')
    print(df_clean.dtypes)
    return df_clean

considerer le temps d'execution de l'embedding dans la logique du pipeline (plus ou moins 20 minutes)

In [ ]:
import os
import psycopg2
from psycopg2.extras import execute_values
import pandas as pd

if 'data_exporter' not in globals():
    from mage_ai.data_preparation.decorators import data_exporter

@data_exporter
def export_data(df, **kwargs):
    """
    Exporte les données vers PostgreSQL et gère les doublons
    """
    try:
        # Connexion à la base de données
        connection = psycopg2.connect(
            host="unruffled_visvesvaraya",  # Nom du service/conteneur PostgreSQL dans docker
            port=5432,  # Le port de connection au container
            database="projet_fil_rouge",  # Le nom de la bdd
            user="postgres",
            password="postgres"
        )

        with connection.cursor() as cur:
            print("Connexion à la base de données")

            # Vérifier les doublons avant l'insertion des données
            print("Vérification des doublons")
            rows_to_drop = []
            for index, row in df.iterrows():
                cur.execute("""
                    SELECT COUNT(*)
                    FROM data_fil_rouge
                    WHERE entreprise = %s
                    AND publication = %s
                    AND poste = %s
                    AND contrat = %s
                    AND ville = %s
                    AND lien = %s
                    AND description_poste = %s;
                """, (row["entreprise"], row["publication"], row["poste"], row["contrat"], row["ville"], row["lien"], row["description_poste"]))
                if cur.fetchone()[0] > 0:
                    print(f"Doublon trouvé et supprimé : {row}")
                    rows_to_drop.append(index)

            # Supprimer les doublons en une seule opération
            if rows_to_drop:
                df = df.drop(rows_to_drop)

            # Préparer les données
            print("Préparation des données")
            tuples = [tuple(x) for x in df.to_numpy()]
            cols = ','.join(list(df.columns))
            query = f"INSERT INTO data_fil_rouge ({cols}) VALUES %s"

            # Insérer les nouvelles données
            if len(tuples) > 0:
                print("Insertion des nouvelles données")
                execute_values(cur, query, tuples)

                # Générer les embeddings pour les nouvelles descriptions de poste
                print("Génération des embeddings")
                cur.execute("""
                    UPDATE data_fil_rouge
                    SET embedding = pgml.embed('intfloat/e5-large', description_poste)
                    WHERE embedding IS NULL;
                """)

                connection.commit()
                print("Nouvelles données insérées!")
            else:
                print("Aucune nouvelle donnée à insérer.")

        # Fermer la connexion
        connection.close()

    except psycopg2.Error as e:
        print(f"Erreur PostgreSQL: {e}")
    except Exception as e:
        print(f"Erreur : {e}")